In [36]:
import numpy as np

from decimal import *
getcontext()
Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999,
        capitals=1, clamp=0, flags=[], traps=[Overflow, DivisionByZero,
        InvalidOperation])

getcontext().prec = 21       # Set a new precision

import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import subprocess
import math

In [37]:
# pequeña funcion que nos servira mas adelante para saber por cuanto debemos multiplicar
# el paso para poder obtener un numero entero que ingresar en el for
def num_after_point(x):
    s = str(x)
    if not '.' in s:
        return 0
    return len(s) - s.index('.') - 1

In [38]:
def chissqr(sigma_s):    # funcion que, dado un sigma sistematico, encuentra el chi cuadrado del grafico de residuos  

# creamos un nuevo archivo .tim en el cual a los errores sumamos este sigma sistematico
    
    timfile = "data_" + str(sigma_s) + ".tim"     # nombre del nuevo archivo .tim, donde hemos sumado el sigma sistematico
    resfile = "residues_" + str(sigma_s) + ".dat" # nombre del archivo que contendra el chi cuadrado
    
    f = open(timfile, "w+")                    # abrimos el nuevo archivo .tim
    print("FORMAT 1" , file = f)               # escibimos la primera linea
             
# ingresamos los datos, con el error aumentado en una cantidad igual a sigma sistematico
    
    for j in range(len(error)):
        newerror = error[j] + sigma_s
        print( 
            str(file[j]) + ' ' + 
            '{:11.6f}'.format(freq[j]) + ' ' + 
            '{:12.6f}'.format(float(MJD1[j])) + MJD2[j] + '   ' + 
            '{:5.3f}'.format(newerror) + '  ' + 
            str(telescope[j]), file = f)
    
    f.close()
    
# llamamos a tempo2
    
    subprocess.call( "tempo2 >" + resfile + " -us -f J0437-4715.par " + timfile , shell=True)

# abrimos el archivo generado por tempo2, donde se encuentra el chicuadrado que buscamos

    chi_sqr_red = np.genfromtxt ( resfile, comments="none", dtype=float, skip_header=19, max_rows=1, usecols=(8) )

# devolvemos la informacion al programa principal

    return chi_sqr_red

In [39]:
# obtenemos el intervalo de sigmas

xmin = np.genfromtxt("interval.txt", comments="none", dtype=float, skip_header=0, max_rows=1, usecols=(2) )
xmax = np.genfromtxt("interval.txt", comments="none", dtype=float, skip_header=1, max_rows=1, usecols=(2) )
step = np.genfromtxt("interval.txt", comments="none", dtype=float, skip_header=2, max_rows=1, usecols=(2) )

print("xmin = " + str(xmin))
print("xmax = " + str(xmax))
print("step = " + str(step))

a = num_after_point(step)           # número de decimales en el paso
xmin_int = int( xmin * (10 ** a) )  # límite inferior entero del intervalo
xmax_int = int( xmax * (10 ** a) )  # límite superior entero del intervalo
step_int = int( step * (10 ** a) )  # paso entero

n = int( (xmax_int-xmin_int)/step_int ) # numero de subintervalos

# print(a)
# print(xmin_int)
# print(xmax_int)
# print(step_int)
# print(n)

# leemos el nombre del archivo .tim

timfile = np.genfromtxt("interval.txt", comments="none", dtype=str, skip_header=3, max_rows=1, usecols=(2))
print(timfile)


xmin = 0.04
xmax = 0.05
step = 0.0001
datav2.tim


In [40]:
# creamos un array para que guarde los sigma sistematicos de prueba y otro para los chi_sqr

trial_sigma = np.empty(n)
chi_sqr = np.empty(n)

In [41]:
# leemos los errores en el TOA del archivo .tim generado por pat

timfile="datav2.tim"
file = np.loadtxt( timfile , skiprows=1 , dtype=str , usecols=(0) )
freq = np.loadtxt( timfile, skiprows=1, dtype=float , usecols=(1) )
MJD1 = np.loadtxt( timfile , skiprows=1 , dtype=np.float64 , usecols=(2) )
MJD2 = np.loadtxt( timfile , skiprows=1 , dtype=str , usecols=(3) )
error = np.loadtxt( timfile , skiprows=1 , dtype=float,  usecols=(4) )
telescope = np.loadtxt( timfile, skiprows=1, dtype=str, usecols=(5) )

#print(error)
#for i in range(0, len(error)):
#    print('{:21.15f}'.format(float(MJD[i])))

In [43]:
# Set-up a period grid with values separated by 0.1 µs between 0 and 1

res = open("resultados.dat", "w+")                                    # abrimos un archivo de salida
i = 0

for x in range(xmin_int, xmax_int, step_int):

    sigma_s = float(x)/ (10. ** a)      # sigma sistematico    

# guardamos la informacion

    trial_sigma[i] = sigma_s
    chi_sqr[i] = chissqr(sigma_s)
    
    print( str(trial_sigma[i]) + ' ' + str(chi_sqr[i]) )                   # escribimos por pantalla  
    res.write(str(trial_sigma[i]) + ' ' + str(chi_sqr[i]))           # escribimos en el archivo de salida
    
    i += 1

# graficamos chi_sqr vs sigma_s

plt.xlabel("sigma_s")
plt.ylabel("chi_sqr_red")
plt.plot(trial_sigma, chi_sqr)
plt.savefig("chicuadrado2.png")
plt.clf()

# Creamos un nuevo array que sea la diferencia entre los chi_sqr obtenidos y 1

diff_chi_sqr = abs ( chi_sqr - 1. )

# Hallamos el máximo chi_sqr mas cercano a 1 y el correspondiente sigma sistematico

print("el chi_sqr mas cercano a 1 es: " + str( chi_sqr [ np.argmin(diff_chi_sqr) ] ) )
print("el correspondiente sigma_s es: " + str( trial_sigma[ np.argmin(diff_chi_sqr) ] ) )
    
    

0.04 1.01107
0.0401 1.01107
0.0402 1.01107
0.0403 1.01107
0.0404 1.01107
0.0405 1.00989
0.0406 1.0078
0.0407 1.0078
0.0408 1.0078
0.0409 1.0078
0.041 1.0078
0.0411 1.0078
0.0412 1.0078
0.0413 1.0078
0.0414 1.0078
0.0415 1.0066
0.0416 1.00455
0.0417 1.00455
0.0418 1.00455
0.0419 1.00455
0.042 1.00455
0.0421 1.00455
0.0422 1.00455
0.0423 1.00455
0.0424 1.00455
0.0425 1.00325
0.0426 1.00131
0.0427 1.00131
0.0428 1.00131
0.0429 1.00131
0.043 1.00131
0.0431 1.00131
0.0432 1.00131
0.0433 1.00131
0.0434 1.00131
0.0435 0.999926
0.0436 0.998087
0.0437 0.998087
0.0438 0.998087
0.0439 0.998087
0.044 0.998087
0.0441 0.998087
0.0442 0.998087
0.0443 0.998087
0.0444 0.998087
0.0445 0.99671
0.0446 0.994879
0.0447 0.994879
0.0448 0.994879
0.0449 0.994879
0.045 0.994879
0.0451 0.994879
0.0452 0.994879
0.0453 0.994879
0.0454 0.994879
0.0455 0.993387
0.0456 0.991687
0.0457 0.991687
0.0458 0.991687
0.0459 0.991687
0.046 0.991687
0.0461 0.991687
0.0462 0.991687
0.0463 0.991687
0.0464 0.991687
0.0465 0.99020